In [1]:
import itertools

In [2]:
def loadDataSet(filename):
    datas = []
    with open(filename, 'r', encoding='utf-8') as fr:
        for line in fr:
            datas.append(line.strip().split(','))
    return datas


def count(items, datas):
    num = 0
    for data in datas:
        if set(items).issubset(set(data)):
            num += 1
    return num


def judgeConnect(items1, items2):
    if len(items1) != len(items2):
        return False
    diff = 0
    for item in items1:
        if item not in items2:
            diff += 1
    if diff == 1:
        return True
    else:
        return False


def judgeSame(itemscur, itemsall):
    for items in itemsall:
        if sorted(itemscur) == sorted(items):
            return True
    return False


def getF1(datas):
    c1 = list(set([item for items in datas for item in items]))
    c1 = [[item] for item in c1]
    f1 = []
    s1 = []  # 支持度
    for c in c1:
        support = count(c, datas) / float(len(datas))
        if support >= min_support:
            f1.append(c)
            s1.append(support)
    return f1, s1


def getFk(fk_1, datas):
    ck = []
    for i in range(len(fk_1)):
        for j in range(i+1, len(fk_1)):
            if judgeConnect(fk_1[i], fk_1[j]):
                items = set(fk_1[i] + fk_1[j])
                if not judgeSame(items, ck):
                    ck.append(list(items))
                    
    fk = []
    sk = []
    for items in ck:
        support = count(items, datas) / float(len(datas))
        if support >= min_support:
            fk.append(items)
            sk.append(support)
    return fk, sk


def generate_fk(datas):
    f_s = {}
    f1, s1 = getF1(datas)
    for f, s in zip(f1, s1):
        f_s[frozenset(f)] = s
    fk, sk = getFk(f1, datas)
    while fk:
        for f, s in zip(fk, sk):
            f_s[frozenset(f)] = s
        fk, sk = getFk(fk, datas)
    return f_s


def generate_rule(f_s):
    rules = []
    for key, value in f_s.items():
        if len(key) >= 2:
            rules.extend(rule(key, f_s, []))
    return rules


def rule(items, f_s, cur_rule):
    for item in itertools.combinations(items, 1):
        if items - frozenset(item) in f_s.keys() and \
                f_s[items] / f_s[items - frozenset(item)] >= min_confidence:
            cur_rule.append((str([items - frozenset(item)]), str(item),
                             f_s[items] / float(f_s[items - frozenset(item)])))
            rule(items - frozenset(item), f_s, cur_rule)
    return cur_rule

In [4]:
#set the minimum support and minimum confidence
min_support = 0.01
min_confidence = 0.5

In [5]:
datas = loadDataSet("..\\Data\\test_data_for_mining")
f_s = generate_fk(datas)
print("Items：{}".format(len(f_s)))
for key, value in f_s.items():
    print("{} : {:.2f}".format(key, value))
rules = generate_rule(f_s)
print("Association rule：{}".format(len(rules)))
for reason, result, conf in rules:
    print("{} ----> {} : {}".format(reason, result, conf))

Items：280
frozenset({'拥堵'}) : 0.51
frozenset({'低中心性'}) : 0.34
frozenset({'2'}) : 0.05
frozenset({'0'}) : 0.20
frozenset({'低流量'}) : 0.42
frozenset({'无效'}) : 0.99
frozenset({'3'}) : 0.22
frozenset({'1'}) : 0.03
frozenset({'高中心性'}) : 0.14
frozenset({'4'}) : 0.25
frozenset({'5'}) : 0.25
frozenset({'高流量'}) : 0.57
frozenset({'畅通'}) : 0.49
frozenset({'爆发'}) : 0.04
frozenset({'低中心性', '拥堵'}) : 0.16
frozenset({'拥堵', '2'}) : 0.01
frozenset({'拥堵', '0'}) : 0.08
frozenset({'拥堵', '低流量'}) : 0.23
frozenset({'无效', '拥堵'}) : 0.51
frozenset({'3', '拥堵'}) : 0.11
frozenset({'拥堵', '高中心性'}) : 0.05
frozenset({'拥堵', '4'}) : 0.17
frozenset({'拥堵', '5'}) : 0.14
frozenset({'高流量', '拥堵'}) : 0.28
frozenset({'爆发', '拥堵'}) : 0.02
frozenset({'低中心性', '2'}) : 0.02
frozenset({'低中心性', '0'}) : 0.06
frozenset({'低中心性', '低流量'}) : 0.09
frozenset({'无效', '低中心性'}) : 0.33
frozenset({'低中心性', '3'}) : 0.10
frozenset({'低中心性', '1'}) : 0.02
frozenset({'低中心性', '4'}) : 0.07
frozenset({'低中心性', '5'}) : 0.07
frozenset({'高流量', '低中心性'}) : 0.25
froze